In [ ]:
import csv
import time
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from nordvpn_switcher import initialize_VPN, rotate_VPN, terminate_VPN

all_restaurants_url = pd.read_csv('newyork.csv', header=None)[0].values
initialize_VPN(save=1,area_input=['complete rotation']) 
user_agent = {'User-agent': 'Mozilla/5.0'}

reviewText = []
reviewRating = []
reviewDate = []
reviewRestaurantName = []

for base_url in all_restaurants_url[238:]:
    counter = 0
    url = base_url +'&sort_by=date_asc'

    while counter < 20:
        try:    
            restaurantPage = requests.get(url)
        except:
            break

        restaurantPageSoup = BeautifulSoup(
            restaurantPage.text, "html.parser"
        )
        reviewsRawList = restaurantPageSoup.findAll(
            "li",
            class_="margin-b5__09f24__pTvws border-color--default__09f24__NPAKY",
        )

        SoupRestaurantName = restaurantPageSoup.find('h1').getText()

        for review in reviewsRawList:
            try:
                reviewText.append(review.find('span', class_="raw__09f24__T4Ezm").getText())
                reviewRating.append(int(review.find("div", class_="five-stars__09f24__mBKym five-stars--regular__09f24__DgBNj display--inline-block__09f24__fEDiJ border-color--default__09f24__NPAKY")['aria-label'][0]))
                reviewDate.append(review.find("span", {"class": "css-chan6m"}).getText())
                reviewRestaurantName.append(SoupRestaurantName)
            except:
                pass

        sleep(1)     

        counter += 1
        url = base_url + '&start=' + str(counter*10) + '&sort_by=date_asc' 
        
        d = {'Review': reviewText, 'Rating': reviewRating, 'Date' : reviewDate, 'Restaurant Name': reviewRestaurantName}
        df = pd.DataFrame(data=d)
        df.to_csv('ny7.csv') #Give different name per city
        
    #execute this code when counter >= 20
    rotate_VPN()
    time.sleep(2)  
    print("VPN has been switched")


In [ ]:
import numpy as np
d = {'Review': reviewText, 'Rating': reviewRating, 'Date' : reviewDate, 'Restaurant Name': reviewRestaurantName}
df = pd.DataFrame(data=d)
df